# 量子アニーリングマシン

In [3]:
import pyqubo
import dimod

# 量子ゲートマシン

# 1ビット演算

## code4.2 1量子ビット

In [2]:
import qiskit
#量子ビットモジュール
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
#古典ビットモジュール
from qiskit import ClassicalRegister
from qiskit import Aer
from qiskit import execute

In [3]:
# 1 量子ビットを定義する
q1 = QuantumRegister(1)
# 1 古典ビットを定義する
c1 = ClassicalRegister(1)
# 結果の個数をカウントする
def get_result(qc, shots=1):
    simulator = Aer.get_backend('qasm_simulator')
    return execute(qc, simulator, shots=shots).result().get_counts(qc)
# 行列を表示する
def get_unitary(qc):
    simulator = Aer.get_backend('unitary_simulator') 
    return execute(qc, simulator).result().get_unitary(qc)

## code4.3 1ビット測定

In [4]:
# 量子回路を定義する
qc1 = QuantumCircuit(q1,c1) 
print(get_unitary(qc1))
# 測定する (量子ビットから古典ビットに) 
qc1.measure(q1,c1)
print(qc1)
# 結果を表示する 
print(get_result(qc1))

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
         ┌─┐
q0_0: |0>┤M├
         └╥┘
 c0_0: 0 ═╩═
            
{'0': 1}


## code4.4 Xゲート
パウリゲートの一つ
<br>
XYZ3つのパウリゲートは角運動量オペレータと同じ交換関係が成り立つ

In [10]:
qc1 = QuantumCircuit(q1,c1) 
# X ゲートを 0 ビット目に適用する 
qc1.x(0)
#量子回路内のユニタリーゲート表示
print(get_unitary(qc1)) 
#ビット測定
qc1.measure(q1,c1) 
#測定結果表示
print(get_result(qc1))
#量子回路表示
print(qc1)

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
{'1': 1}
         ┌───┐┌─┐
q2_0: |0>┤ X ├┤M├
         └───┘└╥┘
 c2_0: 0 ══════╩═
                 


## code4.5 アダマールゲート

In [11]:
qc1 = QuantumCircuit(q1,c1)
# H(アダマール) ゲートを 0 ビット目に適用する 
qc1.h(0)
print(get_unitary(qc1)) 
qc1.measure(q1,c1) 
print(get_result(qc1, 100))
#量子回路表示
print(qc1)

[[ 0.70710678+0.00000000e+00j  0.70710678-8.65956056e-17j]
 [ 0.70710678+0.00000000e+00j -0.70710678+8.65956056e-17j]]
{'1': 44, '0': 56}
         ┌───┐┌─┐
q2_0: |0>┤ H ├┤M├
         └───┘└╥┘
 c2_0: 0 ══════╩═
                 


# 2ビット演算

## code4.6 CXゲート

In [12]:
q2 = QuantumRegister(2)
c2 = ClassicalRegister(2)
qc2 = QuantumCircuit(q2,c2) 
# Xゲートを第一ビットに適用
qc2.x(0)
qc2.measure(q2,c2)
print('before cx') 
print(get_result(qc2))

#上の測定で量子状態が破壊されるのでもう一回初めから
qc2 = QuantumCircuit(q2,c2)
# Xゲートを第一ビットに適用
qc2.x(0)
# 制御NOTゲートを2ビットに適用 cx(controlled_bit, target_bit) 
qc2.cx(0,1)
qc2.measure(q2,c2)
print('after cx') 

#量子回路表示
print(qc2)

before cx
{'01': 1}
after cx
{'11': 1}
         ┌───┐     ┌─┐   
q3_0: |0>┤ X ├──■──┤M├───
         └───┘┌─┴─┐└╥┘┌─┐
q3_1: |0>─────┤ X ├─╫─┤M├
              └───┘ ║ └╥┘
 c3_0: 0 ═══════════╩══╬═
                       ║ 
 c3_1: 0 ══════════════╩═
                         


## code4.7 SWAP

In [13]:
def swap(qc): 
    qc.cx(1,0) 
    qc.cx(0,1) 
    qc.cx(1,0)
    
qc2 = QuantumCircuit(q2,c2) 
qc2.x(0)
qc2.measure(q2,c2) 
print('before swap') 
print(get_result(qc2))
qc2 = QuantumCircuit(q2,c2) 
qc2.x(0)
swap(qc2) 
qc2.measure(q2,c2) 
print('after swap') 
print(get_result(qc2))

#量子回路表示
print(qc2)

before swap
{'01': 1}
after swap
{'10': 1}
         ┌───┐┌───┐     ┌───┐┌─┐   
q3_0: |0>┤ X ├┤ X ├──■──┤ X ├┤M├───
         └───┘└─┬─┘┌─┴─┐└─┬─┘└╥┘┌─┐
q3_1: |0>───────■──┤ X ├──■───╫─┤M├
                   └───┘      ║ └╥┘
 c3_0: 0 ═════════════════════╩══╬═
                                 ║ 
 c3_1: 0 ════════════════════════╩═
                                   


# 3ビット演算

## code4.8 CCXゲート

In [14]:
q3 = QuantumRegister(3)
c3 = ClassicalRegister(3)
qc3 = QuantumCircuit(q3,c3)
qc3.x([0,1,2])
qc3.measure(q3,c3)
print('before ccx')
print(get_result(qc3))

qc3 = QuantumCircuit(q3,c3)
qc3.x([0,1,2])
# ccx(controlled_bit1, controlled_bit2, target_bit) 
qc3.ccx(0,1,2)
qc3.measure(q3,c3)
print('after ccx')
print(get_result(qc3))

print(qc3)

before ccx
{'111': 1}
after ccx
{'011': 1}
         ┌───┐     ┌─┐      
q4_0: |0>┤ X ├──■──┤M├──────
         ├───┤  │  └╥┘┌─┐   
q4_1: |0>┤ X ├──■───╫─┤M├───
         ├───┤┌─┴─┐ ║ └╥┘┌─┐
q4_2: |0>┤ X ├┤ X ├─╫──╫─┤M├
         └───┘└───┘ ║  ║ └╥┘
 c4_0: 0 ═══════════╩══╬══╬═
                       ║  ║ 
 c4_1: 0 ══════════════╩══╬═
                          ║ 
 c4_2: 0 ═════════════════╩═
                            


# 1bitの足し算回路

## code4.9 half_adder

In [15]:
def _half_adder(a, b): 
    # 論理積 (AND)
    c=a&b
    # 排他的論理和 (XOR) 
    s=a^b
    return (c, s)

def classical_half_adder(a, b):
    (c, s) = _half_adder(a, b)
    print('{0} + {1} = {2}{3}'.format(a,b,c,s))
    
def quantum_half_adder(qc, a=0, b=1, c=2): 
    qc.ccx(a,b,c)
    qc.cx(a,b)

def exec_half_adder(a, b):
    q = QuantumRegister(3)
    c = ClassicalRegister(3) 
    qc = QuantumCircuit(q,c)
    if a == 1:
        qc.x(0) 
    if b == 1:
        qc.x(1)
    quantum_half_adder(qc,0,1,2)
    qc.measure(q,c)
    r = list(get_result(qc).keys())[0]
    print("===============================================")
    print('{0} + {1} = {2}{3}'.format(a,b,r[0],r[1]))
    print("(a,b) = ({},{})".format(a,b))
    print(qc)
    print("===============================================")
    
def main_half_adder(): 
    print('Classical Half Adder') 
    classical_half_adder(0, 0) 
    classical_half_adder(0, 1) 
    classical_half_adder(1, 0) 
    classical_half_adder(1, 1) 
    print('Quantum Half Adder') 
    exec_half_adder(0, 0) 
    exec_half_adder(0, 1) 
    exec_half_adder(1, 0) 
    exec_half_adder(1, 1)
    
main_half_adder()

Classical Half Adder
0 + 0 = 00
0 + 1 = 01
1 + 0 = 01
1 + 1 = 10
Quantum Half Adder
0 + 0 = 00
(a,b) = (0,0)
                   ┌─┐   
q5_0: |0>──■────■──┤M├───
           │  ┌─┴─┐└╥┘┌─┐
q5_1: |0>──■──┤ X ├─╫─┤M├
         ┌─┴─┐└┬─┬┘ ║ └╥┘
q5_2: |0>┤ X ├─┤M├──╫──╫─
         └───┘ └╥┘  ║  ║ 
 c5_0: 0 ═══════╬═══╩══╬═
                ║      ║ 
 c5_1: 0 ═══════╬══════╩═
                ║        
 c5_2: 0 ═══════╩════════
                         
0 + 1 = 01
(a,b) = (0,1)
                        ┌─┐   
q6_0: |0>───────■────■──┤M├───
         ┌───┐  │  ┌─┴─┐└╥┘┌─┐
q6_1: |0>┤ X ├──■──┤ X ├─╫─┤M├
         └───┘┌─┴─┐└┬─┬┘ ║ └╥┘
q6_2: |0>─────┤ X ├─┤M├──╫──╫─
              └───┘ └╥┘  ║  ║ 
 c6_0: 0 ════════════╬═══╩══╬═
                     ║      ║ 
 c6_1: 0 ════════════╬══════╩═
                     ║        
 c6_2: 0 ════════════╩════════
                              
1 + 0 = 01
(a,b) = (1,0)
         ┌───┐          ┌─┐   
q7_0: |0>┤ X ├──■────■──┤M├───
         └───┘  │  ┌─┴─┐└╥┘┌─┐
q7_1: |

# 複数ビットの足し算回路

## 練習：ANDゲート

In [16]:
q3 = QuantumRegister(3)
c3 = ClassicalRegister(3)

q_x = [
    [0,0,1,1,2,2],
    [0,0,1,1,2],
    [0,0,1,2,2],
    [0,0,1,2],
    [0,1,1,2,2],
    [0,1,1,2],
    [0,1,2,2],
    [0,1,2]
]

def NOT(q_list):
    qc3 = QuantumCircuit(q3,c3)
    qc3.x(q_list)
    # ccx(controlled_bit1, controlled_bit2, target_bit) 
    qc3.ccx(0,1,2)
    qc3.x([0,0,1,1,2])
    qc3.measure(q3,c3)
    print('after ccx')
    print(get_result(qc3))
    #print(qc3)

for i in range(len(q_x)):
    NOT(q_x[i])


print(q_x)

after ccx
{'100': 1}
after ccx
{'000': 1}
after ccx
{'110': 1}
after ccx
{'010': 1}
after ccx
{'101': 1}
after ccx
{'001': 1}
after ccx
{'011': 1}
after ccx
{'111': 1}
[[0, 0, 1, 1, 2, 2], [0, 0, 1, 1, 2], [0, 0, 1, 2, 2], [0, 0, 1, 2], [0, 1, 1, 2, 2], [0, 1, 1, 2], [0, 1, 2, 2], [0, 1, 2]]


## code4.10 OR

In [17]:
def or_gate(qc, a=0, b=1, r=2): 
    qc.x(a)
    qc.x(b) 
    qc.ccx(a,b,r) 
    qc.x([a,b,r])

def exec_or():
    q = QuantumRegister(3)
    c = ClassicalRegister(3) 
    qc = QuantumCircuit(q,c) 
    qc.x(0)
    qc.measure(q,c) 
    print('before or') 
    print(get_result(qc))
    qc = QuantumCircuit(q,c) 
    qc.x(0) 
    or_gate(qc,0,1,2) 
    qc.measure(q,c) 
    print('after or') 
    print(get_result(qc))
    
exec_or()

before or
{'001': 1}
after or
{'101': 1}


## code4.11 full_adder

In [18]:
def _full_adder(a, b, x):
    (tc, ts) = _half_adder(a, b) 
    (c, s) = _half_adder(x, ts) 
    return (tc | c, s)

def classical_full_adder(a, b, x):
    (c, s) = _full_adder(a, b, x)
    print('{0} + {1} + {2} = {3}{4}'.format(a, b, x, c, s))
    

def quantum_full_adder(qc, a=0, b=1, c1=2, c2=3, ci=4, co=5): 
    quantum_half_adder(qc, a, b, c1)
    quantum_half_adder(qc, b, ci, c2)
    or_gate(qc, c1, c2, co)
    
def exec_full_adder(a, b, x): 
    q = QuantumRegister(6)
    c = ClassicalRegister(6) 
    qc = QuantumCircuit(q,c) 
    if a == 1:
        qc.x(0) 
    if b == 1:
        qc.x(1) 
    if x == 1:
        qc.x(2)
    quantum_full_adder(qc,0,1,2,3,4,5)
    qc.measure(q,c)
    r = list(get_result(qc).keys())[0]
    print('{0} + {1} + {2} = {3}{4}'.format(a,b,x,r[0],r[3]))
    
def main_full_adder(): 
    print('Classical Full Adder') 
    classical_full_adder(0,0,1) 
    classical_full_adder(0,1,1) 
    classical_full_adder(1,1,0) 
    classical_full_adder(1,1,1) 
    print('Quantum Full Adder') 
    exec_full_adder(0,0,1) 
    exec_full_adder(0,1,1) 
    exec_full_adder(1,1,0) 
    exec_full_adder(1,1,1)
    
main_full_adder()

Classical Full Adder
0 + 0 + 1 = 01
0 + 1 + 1 = 10
1 + 1 + 0 = 10
1 + 1 + 1 = 11
Quantum Full Adder
0 + 0 + 1 = 11
0 + 1 + 1 = 11
1 + 1 + 0 = 11
1 + 1 + 1 = 00


# 4.4 グローバーのアルゴリズム

## 4.4.1 2ビットのグローバーのアルゴリズム

### code4.12 探索解の振幅反転2ビット 
探索解の振幅反転を行うゲートを設定する関数"marking2"の第 2 引数の"k"には 2 ビットの値を文字列で入力します。

In [1]:
def marking2(qc, k): 
    if k == '11':
        qc.cz(0,1) 
    elif k == '10':
        qc.x(0) 
        qc.cz(0,1)
        qc.x(0)
    elif k == '01': 
        qc.x(1)
        qc.cz(0,1)
        qc.x(1) 
    else:
        qc.x([0,1]) 
        qc.cz(0,1) 
        qc.x([0,1])

### code4.13 平均値での全振幅反転2ビット
"amp2"は平均値での全振幅反転を行うゲートを設定する関数です。

In [7]:
def amp2(qc): 
    qc.h([0,1]) 
    qc.x([0,1])
    qc.cz(0,1) 
    qc.x([0,1]) 
    qc.h([0,1])

### code4.14 グローバーのアルゴリズム2ビット版

In [8]:
def grover2(k):
    q2 = QuantumRegister(2)
    c2 = ClassicalRegister(2) 
    qc2 = QuantumCircuit(q2,c2) 
    qc2.h([0,1])
    marking2(qc2, k)
    amp2(qc2) 
    qc2.measure(q2,c2) 
    print(get_result(qc2, 100))
    
def exec_grover2():
    print("Grover's Algorithm 2bit")
    grover2('00') 
    grover2('01') 
    grover2('10') 
    grover2('11')
    
exec_grover2()

Grover's Algorithm 2bit
{'00': 100}
{'01': 100}
{'10': 100}
{'11': 100}
